## Q1. What is the estimated depth of a Decision Tree trained (unrestricted) on a one million instance training set?

In [ ]:
'''
m leaves, with log2(m)^3 depth. A binary Decision Tree will end up balanced at the end of training, with one leaf per training instance.
log2(10^6) = 20, maybe a bit more.
'''

## Q2. Is the Gini impurity of a node usually lower or higher than that of its parent? Is it always lower/greater, or is it usually lower/greater?

In [ ]:
"""
It is generally lower than it's parents. If one child is smaller than the other, it is possible for it to have a higher Gini score than its parent
"""

## Q3. Explain if its a good idea to reduce max depth if a Decision Tree is overfitting the training set?

In [ ]:
'''
Yes, since this will constrain and regularize the model
'''

## Q4. Explain if its a  good idea to try scaling the input features if a Decision Tree underfits the training set?

In [ ]:
'''
Scaling the inputs don't matter because a decision tree's output is not affected by scaling or data being centered
'''

## Q5. How much time will it take to train another Decision Tree on a training set of 10 million instances if it takes an hour to train a Decision Tree on a training set with 1 million instances?

In [ ]:
'''
Training time = (n X 10m X log(10m)) / (n x m x log(m)) = 11.7 hours
'''

## Q6. Will setting presort=True speed up training if your training set has 100,000 instances?

In [ ]:
'''
It will actually slow down training. Presorting only speeds up data if the dataset is smaller than a few thousand instances.
'''

## Q7. Follow these steps to train and fine-tune a Decision Tree for the moons dataset:

a. To build a moons dataset, use make moons(n samples=10000, noise=0.4).

b. Divide the dataset into a training and a test collection with train test split().

c. To find good hyperparameters values for a DecisionTreeClassifier, use grid search with cross-validation (with the GridSearchCV class). Try different values for max leaf nodes.

d. Use these hyperparameters to train the model on the entire training set, and then assess its output on the test set. You can achieve an accuracy of 85 to 87 percent.


In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_moons
import pandas as pd
import numpy as np

In [9]:
x, y = make_moons(n_samples=10000, noise=0.4)


In [10]:
x

array([[ 0.5770202 ,  0.77632109],
       [ 0.31480067, -0.19105419],
       [ 1.43665025, -1.17092818],
       ...,
       [ 0.16528027,  0.09155034],
       [ 0.33402528, -0.58401844],
       [-0.96597476,  0.69622997]])

In [11]:
y

array([0, 1, 1, ..., 0, 1, 0])

In [15]:
df = pd.DataFrame(dict(x=x[:,0], y=x[:,1], label=y))

In [18]:
df.shape

(10000, 3)

In [17]:
X = df.drop(columns=['label'])
y = df['label']

In [19]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=44)

In [22]:
dt_model = DecisionTreeClassifier()
dt_model.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [25]:
print(classification_report(y_test, dt_model.predict(x_test)))

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      1497
           1       0.80      0.80      0.80      1503

    accuracy                           0.80      3000
   macro avg       0.80      0.80      0.80      3000
weighted avg       0.80      0.80      0.80      3000



In [30]:
grid_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [50, 60, 80, 90, 100, 120],
    'min_samples_split': range(2, 10, 1),
    'splitter': ['best', 'random'],
    'min_samples_leaf': range(1, 10, 1)
}

gridcv = GridSearchCV(DecisionTreeClassifier(), grid_params, cv=5)
gridcv.fit(x_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                   

In [31]:
gridcv.best_params_

{'criterion': 'gini',
 'max_depth': 50,
 'min_samples_leaf': 7,
 'min_samples_split': 8,
 'splitter': 'random'}

In [32]:
gridcv.best_score_

0.8634285714285713

In [33]:
dt_model2 = DecisionTreeClassifier(criterion='gini', max_depth=50, splitter='random', min_samples_leaf=7, min_samples_split=8)
dt_model2.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=50, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=7, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='random')

In [34]:
print(classification_report(y_test, dt_model2.predict(x_test)))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      1497
           1       0.86      0.83      0.85      1503

    accuracy                           0.85      3000
   macro avg       0.85      0.85      0.85      3000
weighted avg       0.85      0.85      0.85      3000



## Q8. Follow these steps to grow a forest:

         a. Using the same method as before, create 1,000 subsets of the training set, each containing 100 instances chosen at random. You can do this with Scikit-ShuffleSplit Learn's class.

          b. Using the best hyperparameter values found in the previous exercise, train one Decision Tree on each subset. On the test collection, evaluate these 1,000 Decision Trees. These Decision        Trees would likely perform worse than the first Decision Tree, achieving only around 80% accuracy, since they were trained on smaller sets.

         c. Now the magic begins. Create 1,000 Decision Tree predictions for each test set case, and keep only the most common prediction (you can do this with SciPy's mode() function). Over the test collection, this method gives you majority-vote predictions.

         d. On the test range, evaluate these predictions: you should achieve a slightly higher accuracy than the first model (approx 0.5 to 1.5 percent higher). You've successfully learned a Random Forest classifier!


In [72]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100
df = []
rs = ShuffleSplit(n_splits=n_trees, test_size=0.3, random_state=42)
for train_index, test_index in rs.split(x_train):
    x_mini_train = x_train.iloc[train_index]
    y_mini_train = y_train.iloc[train_index]
    df.append((x_mini_train, y_mini_train))

In [73]:
from sklearn.base import clone
forest = [clone(gridcv.best_estimator_) for _ in range(n_trees)]
accuracy = []
for tree, (x_mini_train, y_mini_train) in zip(forest, df):
    tree.fit(x_mini_train, y_mini_train)
    y_pred = tree.predict(x_test)
    accuracy.append(accuracy_score(y_test, y_pred))

np.mean(accuracy)

0.8156123333333334

In [74]:
Y_pred = np.empty([n_trees, len(x_test)])

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(x_test)

In [75]:
from scipy.stats import mode
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [76]:
accuracy_score(y_test, y_pred_majority_votes.ravel())

0.8526666666666667